>Xgboost process data in its own format i.e.Dmatrix<br>
>CatBoot(Categorical Boosting) from company Yandex<br>
>>cat boost works withput hot encoding(without creating dummies)

In [25]:
from catboost import CatBoostClassifier

In [2]:
import pandas as pd
import numpy as np

In [3]:
#hr = pd.read_csv("/Users/darshmac/Documents/cdac/For DBDA/PML/Cases/human-resources-analytics/HR_comma_sep.csv")
hr = pd.read_csv("/kaggle/input/hr-comma-sep/HR_comma_sep.csv")
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [37]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14995 entries, 0 to 14994
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14995 non-null  float64
 1   last_evaluation        14995 non-null  float64
 2   number_project         14995 non-null  int64  
 3   average_montly_hours   14995 non-null  int64  
 4   time_spend_company     14995 non-null  int64  
 5   Work_accident          14995 non-null  int64  
 6   left                   14995 non-null  int64  
 7   promotion_last_5years  14995 non-null  int64  
 8   Department             14995 non-null  object 
 9   salary                 14995 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [30]:
data_types = hr.dtypes == object
data_types[data_types == True]

Department    True
salary        True
dtype: bool

In [21]:
X = hr.drop('left',axis=1)
y = hr['left']

In [22]:
from sklearn.model_selection import StratifiedKFold,GridSearchCV,train_test_split

In [23]:
kflod = StratifiedKFold(n_splits=5, shuffle=True,random_state=23)

In [33]:
cbc = CatBoostClassifier(random_state=23,cat_features=['Department', "salary"])

In [34]:
cbgm = {'n_estimators':[25,50,75,100],
             'learning_rate':np.linspace(0.001,0.7,5),
             'max_depth':[None,2,3,5]}
cbgcv = GridSearchCV(cbc,param_grid=cbgm,cv=kflod,verbose=3,scoring='neg_log_loss')

In [35]:
%%time
cbgcv.fit(X,y)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
0:	learn: 0.6923687	total: 64.2ms	remaining: 1.54s
1:	learn: 0.6915396	total: 69ms	remaining: 794ms
2:	learn: 0.6907662	total: 74.2ms	remaining: 544ms
3:	learn: 0.6899728	total: 79ms	remaining: 415ms
4:	learn: 0.6891675	total: 83.8ms	remaining: 335ms
5:	learn: 0.6883685	total: 88.3ms	remaining: 280ms
6:	learn: 0.6875520	total: 92.6ms	remaining: 238ms
7:	learn: 0.6867320	total: 97ms	remaining: 206ms
8:	learn: 0.6859157	total: 101ms	remaining: 180ms
9:	learn: 0.6850952	total: 105ms	remaining: 158ms
10:	learn: 0.6842885	total: 109ms	remaining: 139ms
11:	learn: 0.6834794	total: 113ms	remaining: 123ms
12:	learn: 0.6826854	total: 118ms	remaining: 109ms
13:	learn: 0.6818795	total: 123ms	remaining: 96.6ms
14:	learn: 0.6810709	total: 128ms	remaining: 85ms
15:	learn: 0.6802665	total: 132ms	remaining: 74ms
16:	learn: 0.6794995	total: 136ms	remaining: 63.9ms
17:	learn: 0.6786986	total: 140ms	remaining: 54.6ms
18:	learn: 0.6778965	total:

KeyError: 'max_depth'

In [36]:
print(cbgcv.best_params_)
print(cbgcv.best_score_)

{'learning_rate': 0.7, 'max_depth': None, 'n_estimators': 100}
-0.06647743245566191
